In [22]:
import create_population
import create_population3
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction

import pandas as pd


import condition_prediction

from train_lightgbm_rank_diff import Trainer_lightgbm_rank_diff 
from train_lightgbm_rank_niti import Trainer_lightgbm_rank_niti
from train_lightgbm_time import Trainer_lightgbm_time
from train_lightgbm_rank_kaiki import Trainer_lightgbm_rank_kaiki

from evaluation_lightgbm_rank_diff import Evaluator_lightgbm_rank_diff
from evaluation_lightgbm_rank_niti import Evaluator_lightgbm_rank_niti
from evaluation_lightgbm_time_kaiki import Evaluator_lightgbm_time_kaiki
from evaluation_lightgbm_rank_kaiki import Evaluator_lightgbm_rank_kaiki
from evaluation_pop import Evaluator_pop

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
%autoreload


In [24]:
# レース結果テーブルの前処理
results_preprocessed = condition_prediction.process_results()
# レース情報テーブルの前処理
race_info_preprocessed = condition_prediction.process_race_info()

In [13]:
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 


# commonディレクトリのパス
COMMON_DATA_DIR = Path("..", "..", "common", "data")
POPULATION_DIR = COMMON_DATA_DIR / "prediction_population"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"


DATA_DIR = Path("..", "data")
OLD_POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)

In [17]:
population_dir= POPULATION_DIR
population_filename = "population.csv"

old_population_dir= OLD_POPULATION_DIR
old_population_filename = "population.csv"
old_results_filename = "results.csv"
old_race_info_filename = "race_info.csv"
old_horse_results_filename = "horse_results.csv"

input_dir = INPUT_DIR

horse_results_filename = "horse_results_prediction.csv"
jockey_leading_filename = "jockey_leading.csv"
trainer_leading_filename = "trainer_leading.csv"
peds_filename = "peds_prediction.csv"
sire_leading_filename = "sire_leading.csv"
output_dir= OUTPUT_DIR
output_filename = "features_prediction.csv"


old_results_condition_filename = "results_prediction.csv"
old_race_info_condition_filename = "race_info_prediction.csv"     

population = pd.read_csv(population_dir / population_filename, sep="\t")
old_results = pd.read_csv(input_dir / old_results_filename, sep="\t")
old_race_info = pd.read_csv(input_dir / old_race_info_filename, sep="\t")
old_population = pd.read_csv(old_population_dir / old_population_filename, sep="\t")
old_horse_results = pd.read_csv(input_dir / old_horse_results_filename, sep="\t")

old_results_condition = pd.read_csv(input_dir / old_results_condition_filename, sep="\t")
old_race_info_condition = pd.read_csv(input_dir / old_race_info_condition_filename, sep="\t")


horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")
jockey_leading = pd.read_csv(input_dir / jockey_leading_filename, sep="\t")
trainer_leading = pd.read_csv(
    input_dir / trainer_leading_filename, sep="\t"
)
peds = pd.read_csv(input_dir / peds_filename, sep="\t")
sire_leading = pd.read_csv(input_dir / sire_leading_filename, sep="\t")
output_dir = output_dir
output_filename = output_filename
htmls = {}
agg_horse_per_group_cols_dfs = {}

In [38]:
race_id="202408060411",  # 予測するレースidを指定
skip_agg_horse=True  # 事前に集計した場合はスキップできる
print("fetching shutuba page html...")
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
# chrome driverをインストール
driver_path = ChromeDriverManager().install()
url = f"https://race.netkeiba.com/race/shutuba.html?race_id={race_id}"
with webdriver.Chrome(service=Service(driver_path), options=options) as driver:
    driver.implicitly_wait(10000)
    driver.get(url)
    htmls[race_id] = driver.page_source
print("fetching shutuba page html...comp")

fetching shutuba page html...
fetching shutuba page html...comp


In [25]:
"""
出馬表ページのhtmlを受け取ると、
「レース結果テーブル」を取得して、学習時と同じ形式に前処理する関数。
"""
html: str
df = pd.read_html(StringIO(html))[0]
df.columns = df.columns.get_level_values(1)
soup = BeautifulSoup(html, "lxml").find("table", class_="Shutuba_Table")
horse_id_list = []
a_list = soup.find_all("a", href=re.compile(r"/horse/"))
for a in a_list:
    horse_id = re.findall(r"\d{10}", a["href"])[0]
    horse_id_list.append(int(horse_id))
df["horse_id"] = horse_id_list
jockey_id_list = []
a_list = soup.find_all("a", href=re.compile(r"/jockey/"))
for a in a_list:
    jockey_id = re.findall(r"\d{5}", a["href"])[0]
    jockey_id_list.append(int(jockey_id))
df["jockey_id"] = jockey_id_list
trainer_id_list = []
a_list = soup.find_all("a", href=re.compile(r"/trainer/"))
for a in a_list:
    trainer_id = re.findall(r"\d{5}", a["href"])[0]
    trainer_id_list.append(int(trainer_id))
df["trainer_id"] = trainer_id_list
# 前処理
df["wakuban"] = df.iloc[:, 0].astype(int)
df["umaban"] = df.iloc[:, 1].astype(int)
df["sex"] = df.iloc[:, 4].str[0].map(sex_mapping)
df["age"] = df.iloc[:, 4].str[1:].astype(int)
df["impost"] = df.iloc[:, 5].astype(float)
# df["weight"] = df.iloc[:, 8].str.extract(r"(\d+)").astype(int)
# df["weight_diff"] = df.iloc[:, 8].str.extract(r"\((.+)\)").astype(int)
df["weight"] = df.iloc[:, 8].astype(str).str.extract(r"(\d+)").astype(float)
df["weight_diff"] = df.iloc[:, 8].astype(str).str.extract(r"\((.+)\)").astype(float)


df["tansho_odds"] = df.iloc[:, 9].astype(float)
df["popularity"] = df.iloc[:, 10].astype(int)
df["race_id"] = int(race_id)
df["n_horses"] = df.groupby("race_id")["race_id"].transform("count")



# 使用する列を選択
df = df[
    [
        "race_id",
        "horse_id",
        "jockey_id",
        "trainer_id",
        # "rank", 
        # "rank_per_horse",                
        # "time",
        # "nobori",
        "umaban",
        "wakuban",
        "tansho_odds",
        "popularity",
        "impost",
        "sex",
        "age",
        "weight",
        "weight_diff",
        "n_horses",
    ] 
]


NameError: name 'html' is not defined

In [41]:
htmls[race_id]
race_type_mapping = race_type_mapping
around_mapping= around_mapping
weather_mapping = weather_mapping
ground_state_mapping = ground_state_mapping
race_class_mapping = race_class_mapping

"""
出馬表ページのhtmlを受け取ると、
「レース情報テーブル」を取得して、学習時と同じ形式に前処理する関数。
"""
info_dict = {}
soup = BeautifulSoup(html, "lxml").find("div", class_="RaceList_Item02")
title = soup.find("h1").text.strip()
divs = soup.find_all("div")
div1 = divs[0].text.replace(" ", "")
info1 = re.findall(r"[\w:]+", div1)
info_dict["race_type"] = race_type_mapping[info1[1][0]]
info_dict["around"] = (
    around_mapping[info1[2][0]] if info_dict["race_type"] != 2 else np.nan
)
info_dict["course_len"] = int(re.findall(r"\d+", info1[1])[0])
info_dict["weather"] = weather_mapping[re.findall(r"天候:(\w+)", div1)[0]]
info_dict["ground_state"] = ground_state_mapping[
    re.findall(r"馬場:(\w+)", div1)[0]
]
# レース階級情報の取得
regex_race_class = "|".join(race_class_mapping)
race_class_title = re.findall(regex_race_class, title)
# タイトルからレース階級情報が取れない場合
race_class = re.findall(regex_race_class, divs[1].text)
if len(race_class_title) != 0:
    info_dict["race_class"] = race_class_mapping[race_class_title[0]]
elif len(race_class) != 0:
    info_dict["race_class"] = race_class_mapping[race_class[0]]
else:
    info_dict["race_class"] = None
info_dict["place"] = int(race_id[4:6])

AttributeError: 'NoneType' object has no attribute 'find'

### 予測当日の処理

In [25]:
# 当日出走馬の過去成績テーブルの前処理_そのままで使えない未加工のデータを加工する
horse_results_preprocessed = preprocessing.process_horse_results(
    input_filename="horse_results_prediction.csv",
    output_filename="horse_results_prediction.csv"
)

In [26]:
# 当日出走馬の血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds(
    input_filename="peds_prediction.csv",
    output_filename="peds_prediction.csv"
)

In [31]:
pfc = PredictionFeatureCreator()
# 過去成績集計は事前に行うことができる
pfc.create_baselog()
pfc.agg_horse_n_races()
pfc.agg_horse_n_races_relative()
pfc.agg_horse_n_races_df

agg_horse_n_races_relative:   0%|          | 0/4 [00:00<?, ?it/s]

,date,race_id,horse_id,rank_1races,rank_per_horse_1races,prize_1races,rank_3races,rank_per_horse_3races,prize_3races,rank_5races,rank_per_horse_5races,prize_5races,rank_10races,rank_per_horse_10races,prize_10races,rank_1000races,rank_per_horse_1000races,prize_1000races
0,2024-11-10,202405050406,2019105148,6.0,0.545455,0.0,8.333333,0.645188,0.000000,7.2,0.584256,24.00,9.100,0.672810,20.000,7.416667,0.529568,68.041667
1,2024-11-10,202405050406,2020110020,7.0,0.583333,0.0,4.333333,0.323611,133.333333,5.4,0.400833,80.00,5.500,0.394306,127.000,5.187500,0.361225,114.875000
2,2024-11-10,202405050406,2021103332,8.0,0.533333,0.0,7.333333,0.469444,18.333333,6.8,0.431667,22.00,6.600,0.415833,41.300,6.818182,0.429167,37.545455
3,2024-11-10,202405050406,2020100406,6.0,0.333333,0.0,6.333333,0.425926,56.666667,6.2,0.446032,58.00,5.700,0.420238,119.000,4.923077,0.359133,139.230769
4,2024-11-10,202405050406,2021105897,12.0,0.923077,0.0,5.000000,0.370192,256.666667,4.6,0.333654,192.60,4.600,0.333654,192.600,4.600000,0.333654,192.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2024-11-10,202403030411,2016104319,3.0,0.187500,705.0,4.666667,0.340278,375.000000,6.4,0.435000,225.00,7.800,0.507626,112.500,6.256410,0.422665,313.492308
134,2024-11-10,202403030411,2021104094,9.0,0.750000,0.0,8.333333,0.580065,537.400000,6.8,0.477585,604.68,5.125,0.375814,709.125,5.125000,0.375814,709.125000
135,2024-11-10,202403030412,2019106520,2.0,0.181818,320.0,2.333333,0.218013,280.000000,5.4,0.404022,168.00,5.400,0.370761,148.000,6.933333,0.456341,112.000000
136,2024-11-10,202403030412,2020103670,8.0,0.500000,0.0,9.333333,0.583333,26.666667,10.2,0.646667,16.00,8.200,0.526178,40.000,7.647059,0.513348,64.588235


In [32]:
# 特徴量の更新
features = pfc.create_features(
    race_id="202408060411",  # 予測するレースidを指定
    date_content_a = "2024年11月10日", #"%Y年%m月%d日"形式で該当レース当日の日付を入れてください
    skip_agg_horse=True  # 事前に集計した場合はスキップできる
)

fetching shutuba page html...
fetching shutuba page html...comp
0    2023
Name: date_year, dtype: int32
running cross_features()...comp
running agg_interval()...comp
running cross_features_2()...comp


agg_cross_encoded:   0%|          | 0/4 [00:00<?, ?it/s]

running cross_features_3()...comp


agg_cross_zizoku_syunpatu:   0%|          | 0/4 [00:00<?, ?it/s]

running cross_features_4()...comp
running cross_features_5()...comp
running cross_features_6()...comp


...

agg_horse_per_race_class:   0%|          | 0/4 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/4 [00:00<?, ?it/s]

agg_horse_per_race_place_len:   0%|          | 0/4 [00:00<?, ?it/s]

agg_horse_per_weather:   0%|          | 0/4 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...
running agg_sire()...
merging all features...
merging all features...comp


In [8]:
pfc.results

,race_id,horse_id,jockey_id,trainer_id,umaban,wakuban,tansho_odds,popularity,impost,sex,age,weight,weight_diff,n_horses
0,202408060411,2020110060,1163,1055,1,1,8.7,2,56.0,1,4,536.0,0.0,17
1,202408060411,2019103588,1150,1020,2,1,35.7,10,56.0,1,5,448.0,-10.0,17
2,202408060411,2019103973,1032,420,3,2,53.0,13,56.0,1,5,444.0,-4.0,17
3,202408060411,2020105626,1174,1110,4,2,25.8,8,56.0,1,4,480.0,-2.0,17
4,202408060411,2020103322,5203,1064,5,3,30.1,9,56.0,1,4,482.0,0.0,17
5,202408060411,2020101024,1180,1098,6,3,126.8,15,56.0,1,4,488.0,6.0,17
6,202408060411,2021105898,5339,1126,7,4,1.9,1,54.0,1,3,472.0,4.0,17
7,202408060411,2020101608,1153,1152,8,4,12.9,6,56.0,1,4,462.0,4.0,17
8,202408060411,2020102873,1157,1120,9,5,39.3,11,56.0,1,4,506.0,-2.0,17
9,202408060411,2019105448,1093,399,10,5,207.5,17,56.0,1,5,464.0,-8.0,17


In [9]:
pfc.race_info

,race_id,race_type,around,course_len,weather,ground_state,race_class,place,race_day_count,race_date_day_count,...,place_season_condition_type,place_season_condition_type_categori,place_course_category,place_course_tough,goal_range,curve,goal_slope,goal_range_100,place_season_type_course_len,lap_type
0,202408060411,1,0,2200,2,0,5,8,11,73870911,...,8321,3,802,3,403,5,1,4.037,8312200,2


In [111]:
import yaml

# YAMLファイルを読み込む
with open("config_lightgbm_niti_odds_removed.yaml", "r") as f:
    config = yaml.safe_load(f)

# YAMLファイルに保存されている特徴量リストを取得
training_features = config.get("features", [])

# トレーニングデータの特徴量リスト
train_columns = training_features
# トレーニングデータと予測データの列順を一致させる
features = features[train_columns]


In [112]:
# トレーニングデータと予測データの列を一致させる
features = features[train_columns]


In [33]:
# オッズ除外モデルでの予測
prediction.predict(
    features,
    model_filename="model_lightgbm_rank_niti_odds_removed.pkl",
    config_filepath="config_lightgbm_niti_odds_removed.yaml"
)

,race_id,umaban,tansho_odds,popularity,mean_fukusho_rate_wakuban,mean_fukusho_rate_umaban,pred
0,202408060411,1,8.7,2,NaN,NaN,0.240350
15,202408060411,16,41.2,12,NaN,NaN,0.126704
12,202408060411,13,12.9,5,NaN,NaN,0.100871
11,202408060411,12,10.4,4,NaN,NaN,0.090324
6,202408060411,7,1.9,1,NaN,NaN,0.076785
14,202408060411,15,195.4,16,NaN,NaN,0.073079
13,202408060411,14,19.1,7,NaN,NaN,0.059858
7,202408060411,8,12.9,6,NaN,NaN,0.058744
3,202408060411,4,25.8,8,NaN,NaN,0.054368
5,202408060411,6,126.8,15,NaN,NaN,0.050776


In [29]:
import yaml

# YAMLファイルを読み込む
with open("config_lightgbm_niti_odds_removed.yaml", "r") as f:
    config = yaml.safe_load(f)

# YAMLファイルに保存されている特徴量リストを取得
training_features = config.get("features", [])

# トレーニングデータの特徴量リスト
train_columns = training_features

# 予測用データの特徴量リスト
prediction_features = features.columns.tolist()

# 差分を確認
missing_features = set(train_columns) - set(prediction_features)
extra_features = set(prediction_features) - set(train_columns)

# 結果を表示
print("トレーニングデータに存在し、予測データに欠けている特徴量:", missing_features)
print("予測データに存在し、トレーニングデータにない特徴量:", extra_features)


トレーニングデータに存在し、予測データに欠けている特徴量: set()
予測データに存在し、トレーニングデータにない特徴量: {'popularity', 'tansho_odds'}


In [12]:
listでやる必要がある
concatしている場合、同じ列がある
colums

NameError: name 'listでやる必要がある' is not defined